# Multipool Google Sentiment Analysis

this function will get data from google_result table
and calculate the sentiment analysis. Result is store in google_result_sentiment

crontab command
* * * * * /home/haviz/multipool/ai-case-study/run-ai-sa-google.sh >> /home/haviz/multipool/ai-sa-google.log



In [1]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import pymysql.cursors
import sqlalchemy as sa
import nltk
import string
import os

from nltk.corpus import stopwords 
from sqlalchemy import create_engine, Column, Integer, String, MetaData, Table
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import pandas as pd

# nltk.download('stopwords')


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Use a pipeline as a high-level helper
from transformers import pipeline
pipe = pipeline("text-classification", model="ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa")

In [2]:
from sqlalchemy.util import deprecations
deprecations.SILENCE_UBER_WARNING = True

def execute_query(query, params=None):
    # Set your PostgreSQL connection parameters
    db_params = {
        'host': '98.98.117.105',
        'port': '5432',
        'database': 'medols',
        'user': 'postgres',
        'password': 'FEWcTB3JIX5gK4T06c1MdkM9N2S8w9pb',
    }

    # Create a SQLAlchemy engine
    engine = create_engine(f"postgresql+psycopg2://{db_params['user']}:{db_params['password']}@{db_params['host']}:{db_params['port']}/{db_params['database']}")

    # Create a metadata object
    metadata = MetaData()

    # Example: Define your table
    your_table = Table('your_table', metadata,
                       Column('id', Integer, primary_key=True),
                       Column('column1', String),
                       Column('column2', String)
                       )

    # Create a session
    Session = sessionmaker(bind=engine)
    session = Session()

    try:
        # Execute the query with optional parameters
        result = session.execute(text(query), params)

        # Check if the query is a SELECT query
        is_select_query = result.returns_rows

        if is_select_query:
            # Fetch the data and return as a Pandas DataFrame
            columns = result.keys()
            fetched_data = result.fetchall()
            df = pd.DataFrame(fetched_data, columns=columns)
            # print("Fetched Data as DataFrame:")
            # print(df)
            return df
        else:
            # Get the number of rows affected for non-SELECT queries
            rows_affected = result.rowcount

            # Commit the changes to the database for non-SELECT queries
            session.commit()

            print(f"Query executed successfully. {rows_affected} rows affected.")
            return rows_affected
    except Exception as e:
        # Rollback changes if there's an error
        session.rollback()
        print(f"Error executing query: {e}")
    finally:
        # Close the session
        session.close()

In [3]:
import pandas as pd
import re
from sqlalchemy import create_engine

def stemming(comment):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in comment:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    return d_clean
    
# function case folding
def casefolding(comment):
    comment = comment.lower()
    comment = comment.strip(" ")
    comment = re.sub(r'[?|$|.|!_:")(-+,]','',comment)
    return comment

def clean_up_tag(comment):
    x_ret = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",comment).split())
    return x_ret

# Text Preprocessing, 
def text_preproc(strIn):
    # case folding
    strOut = strIn.lower()
    
    # remove numbers
    strOut = re.sub(r"\d+", "", strOut)
    
    # remote punctuation
    strOut = strOut.translate(str.maketrans("","",string.punctuation))
    
    # remove whitspace
    strOut = strOut.strip()
    
    # 
    strOut = re.sub('\s+',' ',strOut)
    return strOut

In [4]:
def process_by_df(source_ds, col_name):
    from tqdm import tqdm
    pd.options.mode.chained_assignment = None 
    
    # trim to 2500 character
    source_ds[col_name] = source_ds[col_name].str.slice(0,2000)

    # skip stemming
    source_ds['stemmed'] = source_ds[col_name].apply(clean_up_tag)
    source_ds['stemmed'] = source_ds['stemmed'].apply(casefolding)
    source_ds['stemmed'] = source_ds['stemmed'].apply(text_preproc)

    sw = stopwords.words('indonesian')
    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)
    sw = stopwords.words('indonesian')

    #tokenized
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: nltk.word_tokenize(row['stemmed']), axis=1)

    # apply stopword removal
    source_ds['tokenized_tweet'] = source_ds.apply(lambda row: {w for w in row['tokenized_tweet'] if not w in sw}, axis=1)

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay

    # Create a new list and insert each element from the original list
    from tqdm import tqdm

    list_text = source_ds['stemmed'].tolist()
    new_list = []
    for i in tqdm(range( len(list_text) )):
        # print(len(list_text[i]))
        try:
            if len(list_text[i]) > 0:
                res = pipe( list_text[i] )
                new_list.append(res)
            else:
                res = pipe( 'kalimat netral' )
                new_list.append(res)
                zip
        except Exception as e:
            # Handle the exception (error) here
            print(f"Error processing value at index {i}: {list_text[i]}")
            print(f"Error message: {str(e)}")
            res = pipe( 'kalimat netral' )
            new_list.append(res)      
            
            continue

    # Notes on return sentiment values
    # 1 >> positif >> stay
    # 2 >> negatif >> convert to -1
    # 0 >> netral >> stay
    # create a list of our conditions
    for i in range(0, len(new_list)):
        # print(new_list[i][0]['label'])
        if new_list[i][0]['label'] == 'Positive':
            new_list[i][0]['Value'] = 1
        elif new_list[i][0]['label'] == 'Negative':
            new_list[i][0]['Value'] = -1
        elif new_list[i][0]['label'] == 'Neutral':
            new_list[i][0]['Value'] = 0

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']

    for index, row in source_ds.iterrows():
        source_ds.at[index,'Prediction'] = new_list[index][0]['label']
        source_ds.at[index,'Score'] = new_list[index][0]['score']
        #source_ds.at[index,'Score'] = new_list[index][0]['Value']
        
    return source_ds


In [5]:
# query 400 rows of data query data from table
select_query = "select * \
from news_national_medol gr \
where id not in (select news_national_medol_id from news_national_medol_sentiment) \
order by id desc limit 500"

source_ds = execute_query(select_query)

if len(source_ds) == 0:
    # get out, nothing to do
    print('Zero jobs, quitting now')
    quit()
    
# source_ds.head()

In [6]:
ds_title = source_ds
res_ds = process_by_df(ds_title,'title')

  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 1/207 [00:00<02:13,  1.54it/s]

  1%|          | 2/207 [00:01<02:12,  1.55it/s]

  1%|▏         | 3/207 [00:01<02:07,  1.60it/s]

  2%|▏         | 4/207 [00:02<01:48,  1.87it/s]

...

 97%|█████████▋| 200/207 [00:18<00:00, 11.94it/s]

 98%|█████████▊| 202/207 [00:19<00:00, 12.79it/s]

 99%|█████████▊| 204/207 [00:19<00:00, 12.19it/s]

100%|█████████▉| 206/207 [00:19<00:00, 12.47it/s]

100%|██████████| 207/207 [00:19<00:00, 10.64it/s]

In [7]:
res_ds.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,publish_date_convert,calculation_mark,stemmed,tokenized_tweet,Prediction,Score
0,67267,Pihak Gus Muhdlor Hormati Putusan Hakim yang T...,"JAKARTA, KOMPAS.com - Tim hukum Bupati nonakti...","05/06/2024, 20:29 WIB",Kompas.com,https://www.kompas.com/,https://nasional.kompas.com/read/2024/06/05/20...,2024-06-05 23:26:37.042359,21,False,"Adinda Putri Kintamani Nugraha,",Icha Rastika,False,2024-06-05 20:29:00,1,pihak gus muhdlor hormati putusan hakim yang t...,"{praperadilan, gugatan, muhdlor, gus, hakim, h...",Neutral,0.999694
1,67266,Rangkaian KRL Manggarai-Kampung Bandan Jadi Sa...,"JAKARTA, KOMPAS.com - Viral di media sosial se...","05/06/2024, 20:26 WIB",Kompas.com,https://www.kompas.com/,https://megapolitan.kompas.com/read/2024/06/05...,2024-06-05 23:26:37.034978,21,False,"Firda Janati,",Irfan Maullana,False,2024-06-05 20:26:00,1,rangkaian krl manggarai kampung bandan jadi sa...,"{aksi, kampung, sasaran, manggarai, bandan, kr...",Neutral,0.999210
2,67265,"Trotoar di Pulogadung Sempit, Warga Terpaksa J...","JAKARTA, KOMPAS.com - Warga bernama Aisyah (25...","05/06/2024, 20:20 WIB",Kompas.com,https://www.kompas.com/,https://megapolitan.kompas.com/read/2024/06/05...,2024-06-05 23:26:37.025861,21,False,"Ryan Sara Pratiwi,",Ambaranie Nadia Kemala Movanita,False,2024-06-05 20:20:00,1,trotoar di pulogadung sempit warga terpaksa ja...,"{trotoar, warga, pulogadung, terpaksa, sepeda,...",Neutral,0.992453
3,67264,"Status Gunung Ibu Masih Awas, Warga Diminta Te...","JAKARTA, KOMPAS.com - Badan Nasional Penanggu...","05/06/2024, 20:15 WIB",Kompas.com,https://www.kompas.com/,https://nasional.kompas.com/read/2024/06/05/20...,2024-06-05 23:26:37.017648,21,False,"Tria Sutrisna,",Ardito Ramadhan,False,2024-06-05 20:15:00,1,status gunung ibu masih awas warga diminta tet...,"{warga, mengungsi, awas, gunung, status}",Neutral,0.999681
4,67263,"Siswi SD Korban ""Bullying"" di Depok Dikenal se...","DEPOK, KOMPAS.com - A (12), korban perundungan...","05/06/2024, 20:08 WIB",Kompas.com,https://www.kompas.com/,https://megapolitan.kompas.com/read/2024/06/05...,2024-06-05 23:26:37.010022,21,False,"Dinda Aulia Ramadhanty,",Irfan Maullana,False,2024-06-05 20:08:00,1,siswi sd korban bullying di depok dikenal seba...,"{yatim, dikenal, sd, depok, pendiam, siswi, an...",Neutral,0.795077


In [8]:
# record result
sentiment_class = 0

# INSERT INTO public.news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES( 0, 0, 0);

for index, row in res_ds.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "INSERT INTO news_national_medol_sentiment (sentiment_category, score, news_national_medol_id) VALUES(%s, %s, %s);" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


...

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


In [9]:
ds_content = source_ds
res_ds_content = process_by_df(ds_content,'content')

  0%|          | 0/207 [00:00<?, ?it/s]

  0%|          | 1/207 [00:00<00:57,  3.58it/s]

  1%|          | 2/207 [00:00<00:54,  3.78it/s]

  1%|▏         | 3/207 [00:00<00:54,  3.72it/s]

  2%|▏         | 4/207 [00:01<00:55,  3.66it/s]

...

 99%|█████████▊| 204/207 [01:13<00:00,  4.63it/s]

 99%|█████████▉| 205/207 [01:13<00:00,  4.50it/s]

100%|█████████▉| 206/207 [01:14<00:00,  4.95it/s]

100%|██████████| 207/207 [01:14<00:00,  4.81it/s]

100%|██████████| 207/207 [01:14<00:00,  2.79it/s]

In [10]:
res_ds_content.head()

,id,title,content,publish_date,publisher,source,search_url,created_date,national_medol_id,is_polri,reporter,editor,is_deleted,publish_date_convert,calculation_mark,stemmed,tokenized_tweet,Prediction,Score
0,67267,Pihak Gus Muhdlor Hormati Putusan Hakim yang T...,"JAKARTA, KOMPAS.com - Tim hukum Bupati nonakti...","05/06/2024, 20:29 WIB",Kompas.com,https://www.kompas.com/,https://nasional.kompas.com/read/2024/06/05/20...,2024-06-05 23:26:37.042359,21,False,"Adinda Putri Kintamani Nugraha,",Icha Rastika,False,2024-06-05 20:29:00,1,jakarta kompas com tim hukum bupati nonaktif s...,"{tindak, kuasa, praperadilan, selatan, diputus...",Neutral,0.999541
1,67266,Rangkaian KRL Manggarai-Kampung Bandan Jadi Sa...,"JAKARTA, KOMPAS.com - Viral di media sosial se...","05/06/2024, 20:26 WIB",Kompas.com,https://www.kompas.com/,https://megapolitan.kompas.com/read/2024/06/05...,2024-06-05 23:26:37.034978,21,False,"Firda Janati,",Irfan Maullana,False,2024-06-05 20:26:00,1,jakarta kompas com viral di media sosial seuba...,"{kriminal, line, diduga, menggambar, tindak, m...",Neutral,0.990927
2,67265,"Trotoar di Pulogadung Sempit, Warga Terpaksa J...","JAKARTA, KOMPAS.com - Warga bernama Aisyah (25...","05/06/2024, 20:20 WIB",Kompas.com,https://www.kompas.com/,https://megapolitan.kompas.com/read/2024/06/05...,2024-06-05 23:26:37.025861,21,False,"Ryan Sara Pratiwi,",Ambaranie Nadia Kemala Movanita,False,2024-06-05 20:20:00,1,jakarta kompas com warga bernama aisyah mengat...,"{timur, tenang, dirasa, padat, pembenahan, tak...",Negative,0.971745
3,67264,"Status Gunung Ibu Masih Awas, Warga Diminta Te...","JAKARTA, KOMPAS.com - Badan Nasional Penanggu...","05/06/2024, 20:15 WIB",Kompas.com,https://www.kompas.com/,https://nasional.kompas.com/read/2024/06/05/20...,2024-06-05 23:26:37.017648,21,False,"Tria Sutrisna,",Ardito Ramadhan,False,2024-06-05 20:15:00,1,jakarta kompas com badan nasional penanggulang...,"{rawan, awan, sungi, tebal, meletus, status, p...",Neutral,0.999413
4,67263,"Siswi SD Korban ""Bullying"" di Depok Dikenal se...","DEPOK, KOMPAS.com - A (12), korban perundungan...","05/06/2024, 20:08 WIB",Kompas.com,https://www.kompas.com/,https://megapolitan.kompas.com/read/2024/06/05...,2024-06-05 23:26:37.010022,21,False,"Dinda Aulia Ramadhanty,",Irfan Maullana,False,2024-06-05 20:08:00,1,depok kompas com a korban perundungan atau bul...,"{terlibat, sekolah, berdiam, diduga, sedih, ka...",Negative,0.994939


In [11]:
# record result
sentiment_class = 0

for index, row in res_ds_content.iterrows():
    if row['Prediction'] == 'Positive':
        sentiment_class = 1
    elif row['Prediction'] == 'Negative':
        sentiment_class = -1
    else:
        sentiment_class = 0
        
    sql = "update news_national_medol_sentiment set sentiment_category_content = %s, score_content = %s where news_national_medol_id = %s" % (sentiment_class,row['Score'],row['id']) 
    # print(sql)
    execute_query(sql)

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


...

Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.


Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
Query executed successfully. 1 rows affected.
